# Talking facts: Connecting to a knowledge base

### 🧰 Defining a Tool Function
- We saw in the early example a proof of concept of a customer support agent that welcomes users.
- In ADK, a **tool** is an external function or utility that an agent can use to assist in answering a user’s question or completing a task.
![image-4](images/image-4.png)
- To add this capability, we first need a function that the agent can use as a tool.
- When creating the `LlmAgent`, we provide an additional parameter `tools=[faq_tool]` to register our FAQ lookup function as a tool the agent can use.
- We then create our new support agent and run it.

## ❗️ Note: Run the **hidden cells** below to initialize the agent, before running the rest of the code. ❗️ 

In [1]:
!pip install protobuf==5.28.1 google-adk==1.0.0 litellm -q -q

In [2]:
import importlib
importlib.invalidate_caches()

In [3]:
import os
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
import os

os.environ["OPENAI_API_BASE"]="http://localhost:11434/v1"

In [4]:
from google.adk.tools import FunctionTool
from google.genai import types
from google import genai
from google.adk.models.lite_llm import LiteLlm
import litellm 

AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")
APP_NAME = "adk_course_app"
USER_ID = "user_123"
SESSION_ID = "support_session"

In [5]:
# Define the FAQ knowledge base
FAQ_DATA = {
    "return policy": "You can return items within 30 days of purchase.",
    "hours": "Our support team is available from 9am to 5pm, Monday to Friday.",
    "contact": "You can reach support at help@example.com."
}

def lookup_faq(question: str) -> str:
    faq_text = "\n".join(f"- {k}: {v}" for k, v in FAQ_DATA.items())
    prompt = (
        f"You are a helpful assistant. Here is a list of FAQs:\n\n{faq_text}\n\n"
        f"User question: \"{question}\". "
        f"Reply with the best match or say you don't know."
    )
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

In [6]:
# Wrap the tool
faq_tool = FunctionTool(func=lookup_faq)

support_agent = LlmAgent(
    name="SupportAgent",
    description="An agent that answers users' questions based on a set of FAQs.",
    instruction="Use the FAQ tool to help answer customer questions.",
    model=AGENT_MODEL,
    tools=[faq_tool]
)

In [7]:
# Set up session service and runner
session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=support_agent, app_name=APP_NAME, session_service=session_service)

# Define and call the agent asynchronously
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            break

    print(f"<<< Agent Response: {final_response_text}")

# Run the agent
await call_agent_async("What is your return policy?")


>>> User Query: What is your return policy?
<<< Agent Response: Our return policy allows you to return items within 30 days of purchase.


- With this tool-enabled agent, we’ve introduced a best practice: **augmenting AI agents with tools for interaction with specific information**.